
dataset path => "/media/HDD/PaperPlateQC-Dataset"


In [1]:
import os
import glob
import pandas as pd 
import numpy as np
import json
import ast

In [2]:
def images_table(file):
    f = file
    images_table = []
    for i in range(len(f["images"])):
        id_ = f["images"][i]['id']
        width = f["images"][i]['width']
        height = f["images"][i]['height']
        file_name = f["images"][i]['file_name']
        images_table.append({"image_id": id_,"width": width,"height": height, "file_name": file_name})
    return images_table

def annotations_table(file):
    f = file
    annotations_table = []
    for i in range(len(f["annotations"])):
        image_id = f["annotations"][i]['image_id']
        area = f["annotations"][i]['area']
        bbox = f["annotations"][i]['bbox']
        annotations_table.append({"image_id": image_id,"area": area, "bbox": bbox})
    return annotations_table

# 🍁 Open and Manage .json annotation file

> /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/Json/

In [ ]:
import os  
os.chdir('/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/Json/')  
!ls

In [ ]:
file = open('./instances_Train1.json')
file = json.loads(file.read())

images_table_ = pd.DataFrame(images_table(file))
annotations_table_ = pd.DataFrame(annotations_table(file))
result1 = pd.merge(images_table_, annotations_table_, on="image_id")
result1['Split'] = 'Train'
result1['set'] = 'set1'
print(result1.shape)
result1.head()

In [ ]:
file = open('./instances_Train2.json')
file = json.loads(file.read())

images_table_ = pd.DataFrame(images_table(file))
annotations_table_ = pd.DataFrame(annotations_table(file))
result2 = pd.merge(images_table_, annotations_table_, on="image_id")
result2['Split'] = 'Train'
result2['set'] = 'set2'
print(result2.shape)
result2.head()

In [ ]:
file = open('./instances_Test3.json')
file = json.loads(file.read())

images_table_ = pd.DataFrame(images_table(file))
annotations_table_ = pd.DataFrame(annotations_table(file))
result3 = pd.merge(images_table_, annotations_table_, on="image_id")
result3['Split'] = 'Test'
result3['set'] = 'set3'
print(result3.shape)
result3.head()

In [ ]:
result = result1.append(result2)
result = result.append(result3)
print(result.shape)
result.head()

In [ ]:
list(result['bbox'])[1]

In [ ]:
xmin_, ymin_, xmax_, ymax_ = [], [], [], []

for j in range(len(result)):
    bbox = list(result['bbox'])[j] 
    ymin = bbox[1]
    ymax = bbox[1] + bbox[3]
    xmin = bbox[0]
    xmax = bbox[0] + bbox[2]
    xmin_.append(xmin)
    ymin_.append(ymin)
    xmax_.append(xmax)
    ymax_.append(ymax)
    
result['xmin'] = xmin_
result['ymin'] = ymin_
result['xmax'] = xmax_
result['ymax'] = ymax_

In [ ]:
print(result.shape)
result.head()

In [ ]:
result.groupby(['set'])['file_name'].count().reset_index()

In [ ]:
result.groupby(['Split'])['file_name'].count().reset_index()

# 🍁 Manage images path


> /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/Images

In [ ]:
import os  
os.chdir('/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/Images/')  
!ls

In [ ]:
path = '/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/Images/'
file_name = result['file_name'].tolist()
set_ = result['set'].tolist()

imgs = []
for i in range(len(file_name)):
    img = f'{path}{set_[i]}/{file_name[i]}'
    imgs.append(img)
    print(img)
    
result['img_path'] = imgs

In [ ]:
path_img = result['img_path'].tolist()
# os.chdir('/media/HDD/dataset_cctv_plate/data_defect/Defect_13_Feb_25/all_defect_img/')

import matplotlib.pyplot as plt  
import matplotlib.image as mpimg  
import os  
import random  

# Randomly select a few images  
selected_imgs = random.sample(path_img, 9)  


# Create a figure with nine subplots  
fig, ax = plt.subplots(3, 3, figsize=(15, 15))  

# Display images on the subplots  
for i, img in enumerate(selected_imgs):  
    img_data = mpimg.imread(img)  
    ax[i // 3, i % 3].imshow(img_data)  
    ax[i // 3, i % 3].set_title(os.path.basename(img))  

# Layout so plots do not overlap  
fig.tight_layout()  

# Show the plot  
plt.show()

In [ ]:
selected_imgs

In [ ]:
import pandas as pd  
import matplotlib.pyplot as plt  

f = 6

# Define the image path for plotting  
image_path = selected_imgs[f]  # Update with your image path  
tb = result[result['img_path'] == image_path]  

# Load the image  
img = plt.imread(image_path)  

# Plotting the image  
plt.figure(figsize=(10, 10))  
plt.imshow(img)  

# Iterate over bounding boxes  
for index in range(len(tb)):  
    bbox = tb['bbox'].iloc[index]  # Directly access the list  
    xmin = bbox[0]  
    ymin = bbox[1]  
    width = bbox[2]  
    height = bbox[3]  
    
    # Calculate the maximum x and y coordinates  
    xmax = xmin + width  
    ymax = ymin + height  

    # Draw the rectangle  
    plt.gca().add_patch(plt.Rectangle((xmin, ymin), width, height,  
                                       edgecolor='red', facecolor='none', linewidth=1))  

plt.title('Image with Bounding Boxes')  
plt.axis('off')  # Hide the axis  
plt.show()  

In [ ]:
# height,width,_ =  img.shape
# print(height,width)

# result['height'] = height
# result['width'] = width

In [ ]:
result.to_csv('/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/CSV/Annotation_boxes_Factory_plate_set1-3.csv')